In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics -q

In [15]:
from pathlib import Path
import time
import cv2
import numpy as np
from ultralytics import YOLO
import os

separation_seg = YOLO('/content/drive/MyDrive/Prescription Sorting/text_line_segmentation_best.pt')

def roiSegmentation(image, save_path):
    res = separation_seg.predict(source=image, conf=0.25)
    for r in res:
        img = np.copy(r.orig_img)
        img_name = Path(r.path).stem

        for ci, c in enumerate(r):
            label = c.names[c.boxes.cls.tolist().pop()]
            if c.masks is not None and c.masks.xy is not None:  # Check if masks are available
                contour = c.masks.xy.pop().astype(np.int32).reshape(-1, 1, 2)
                b_mask = np.zeros(img.shape[:2], np.uint8)
                _ = cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED)
                mask3ch = cv2.cvtColor(b_mask, cv2.COLOR_GRAY2BGR)
                isolated = cv2.bitwise_and(mask3ch, img)

                x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32)
                iso_crop = isolated[y1:y2, x1:x2]
                # segmented_file_name = f'{img_name}{label}-{ci}.png'
                segmented_file_name = f'{y1}.png'
                segmented_file_name = os.path.join(save_path, segmented_file_name)
                _ = cv2.imwrite(segmented_file_name, iso_crop)
            else:
                print("No contour masks found for", img_name, label)
            # cv2.imshow("window", iso_crop)
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()
    # return res


# image_list = os.listdir("/content/drive/MyDrive/Prescription Sorting/image")
# print(image_list)
presp_text = "/content/drive/MyDrive/Prescription Sorting/PrespText"

# for image in image_list:
#   print(image)
#   image_name = os.path.join("/content/drive/MyDrive/Prescription Sorting/image", image)
#   print(image_name)
#   roiSegmentation(image_name, presp_text)
#   print(image, "-- Completed")
#   break

roiSegmentation("/content/drive/MyDrive/Prescription Sorting/image/Scanned_20231214-1132_page-0007.jpg", presp_text)


image 1/1 /content/drive/MyDrive/Prescription Sorting/image/Scanned_20231214-1132_page-0007.jpg: 640x512 9 Liness, 81.4ms
Speed: 4.1ms preprocess, 81.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 512)


In [16]:
# res


In [ ]:
masks.xy
masks.xyn

In [34]:
from pathlib import Path
import cv2
import numpy as np
from ultralytics import YOLO
import os

# Load the YOLO model for text line segmentation
separation_seg = YOLO('/content/drive/MyDrive/Prescription Sorting/text_line_segmentation_best.pt')

def roiSegmentation(image, save_path):
    # Predict text line segmentation on the input image
    res = separation_seg.predict(source=image, conf=0.25)
    for r in res:
        img = np.copy(r.orig_img)
        img_name = Path(r.path).stem

        # Sort contours based on y-coordinate in ascending order
        contours_sorted = sorted(r, key=lambda x: x.boxes.xyxy[0][1])

        for ci, c in enumerate(contours_sorted):
            label = c.names[c.boxes.cls.tolist().pop()]
            if c.masks is not None and c.masks.xy is not None:  # Check if masks are available
                contour = c.masks.xy.pop().astype(np.int32).reshape(-1, 1, 2)
                b_mask = np.zeros(img.shape[:2], np.uint8)
                _ = cv2.drawContours(b_mask, [contour], -1, (255, 255, 255), cv2.FILLED)
                mask3ch = cv2.cvtColor(b_mask, cv2.COLOR_GRAY2BGR)
                isolated = cv2.bitwise_and(mask3ch, img)

                x1, y1, x2, y2 = c.boxes.xyxy.cpu().numpy().squeeze().astype(np.int32)
                iso_crop = isolated[y1:y2, x1:x2]
                # Formulate the segmented file name based on y-coordinate
                segmented_file_name = f'{y1}.png'
                segmented_file_name = os.path.join(save_path, segmented_file_name)
                _ = cv2.imwrite(segmented_file_name, iso_crop)
            else:
                print("No contour masks found for", img_name, label)

# Define the path to save the segmented images
presp_text = "/content/drive/MyDrive/Prescription Sorting/PrespText"

# Call roiSegmentation function with the input image and save path
roiSegmentation("/content/Madinah-Medical-20.jpg", presp_text)



image 1/1 /content/Madinah-Medical-20.jpg: 640x448 10 Liness, 64.2ms
Speed: 2.8ms preprocess, 64.2ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 448)


In [33]:
# Delete File

import os

# Specify the directory containing the files
directory = '/content/drive/MyDrive/Prescription Sorting/PrespText'

# Get a list of all files in the directory
files = os.listdir(directory)

# Iterate over each file and delete it
for file in files:
    file_path = os.path.join(directory, file)
    if os.path.isfile(file_path):
        os.remove(file_path)
    else:
        print(f"Skipping {file_path} as it is not a file.")


In [35]:
list = os.listdir('/content/drive/MyDrive/Prescription Sorting/PrespText')
print(list)

['311.png', '340.png', '341.png', '363.png', '364.png', '394.png', '415.png', '458.png', '485.png', '535.png']
